In [9]:
import pandas as pd
import numpy as np
import pickle
import math
import time
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split, cross_validate, RandomizedSearchCV, StratifiedShuffleSplit
from sklearn.neural_network import MLPClassifier
from sklearn.metrics import make_scorer, roc_auc_score, accuracy_score, average_precision_score, f1_score
from scipy.stats import uniform as sp_randFloat
from scipy.stats import randint as sp_randInt
import matplotlib.pyplot as plt
from IPython.display import display
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

In [21]:
dataset1_trainval_df = pd.read_csv("dataset1/processed_trainval_data.csv")
dataset1_test_df = pd.read_csv("dataset1/processed_test_data.csv")
dataset1_df = pd.concat([dataset1_trainval_df, dataset1_test_df])
display(dataset1_df.shape)
display(dataset1_df.head())

(1941, 28)

,X_Minimum,Square_Index,Length_of_Conveyer,Y_Perimeter,Edges_Y_Index,Maximum_of_Luminosity,X_Perimeter,Orientation_Index,Luminosity_Index,Empty_Index,Log_Y_Index,Outside_Global_Index,Minimum_of_Luminosity,Outside_X_Index,Edges_X_Index,Steel_Plate_Thickness,Y_Maximum,Edges_Index,Pixels_Areas,SigmoidOfAreas,LogOfAreas,Sum_of_Luminosity,TypeOfSteel_A400,TypeOfSteel_A300,Y_Minimum,Log_X_Index,X_Maximum,label
0,0.557185,0.579812,0.234568,0.000606,1.000000,0.481481,0.000670,0.709991,0.585558,0.264859,0.253411,1.0,0.566502,0.004231,0.774554,0.519231,0.065121,0.595458,0.000400,0.079682,0.306859,0.000648,1.0,0.0,0.065121,0.196206,0.557636,0
1,0.000000,0.109307,0.231041,0.006556,1.000000,0.259259,0.008711,0.945327,0.352651,0.547410,0.488224,1.0,0.251232,0.010064,0.138088,0.115385,0.051667,0.000000,0.005306,0.999886,0.534254,0.005177,0.0,1.0,0.051659,0.304749,0.005851,0
2,0.024047,0.955430,0.305115,0.025784,0.336801,0.490741,0.072174,0.477531,0.552651,0.521665,0.525512,0.0,0.108374,0.146157,0.228287,0.000000,0.075784,0.058883,0.104118,1.000000,0.798800,0.159364,1.0,0.0,0.075772,0.705600,0.127560,0
3,0.022874,0.973480,0.308642,0.017024,0.464376,0.490741,0.047669,0.513088,0.553870,0.432673,0.512316,1.0,0.152709,0.119053,0.285714,0.000000,0.113532,0.055868,0.087172,1.000000,0.782989,0.133714,1.0,0.0,0.113521,0.674083,0.107080,0
4,0.687977,0.573359,0.222222,0.001377,1.000000,0.393519,0.001723,0.713219,0.492566,0.358618,0.332261,1.0,0.379310,0.010979,0.746347,0.038462,0.082096,0.245076,0.001677,0.593871,0.432261,0.002284,0.0,1.0,0.082095,0.315567,0.692803,0


In [23]:
dataset1_df.to_csv("dataset1/processed_full_data.csv", index=False)

In [22]:
dataset2_trainval_df = pd.read_csv("dataset2/processed_trainval_data.csv")
dataset2_test_df = pd.read_csv("dataset2/processed_test_data.csv")
dataset2_df = pd.concat([dataset2_trainval_df, dataset2_test_df])
display(dataset2_df.shape)
display(dataset2_df.head())

(1473, 22)

,media_exposure,wife_religion,num_children_born,wife_working,wife_age,wife_education_1,wife_education_2,wife_education_3,wife_education_4,husband_education_1,husband_education_2,husband_education_3,husband_education_4,husband_occupation_1,husband_occupation_2,husband_occupation_3,husband_occupation_4,standard_of_living_index_1,standard_of_living_index_2,standard_of_living_index_3,standard_of_living_index_4,label
0,0,0,2,1,28,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1
1,1,1,1,0,27,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0
2,0,1,2,1,28,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,1
3,0,1,3,1,24,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,1
4,0,0,4,1,35,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,1


In [24]:
dataset2_df.to_csv("dataset2/processed_full_data.csv", index=False)